## Install libraries and download pretrained model

In [ ]:
!pip install mmcv-full

In [ ]:
!git clone https://github.com/open-mmlab/mmaction2.git

In [2]:
!wget -O slowfast.pth https://download.openmmlab.com/mmaction/recognition/slowfast/slowfast_r50_256p_4x16x1_256e_kinetics400_rgb/slowfast_r50_256p_4x16x1_256e_kinetics400_rgb_20200728-145f1097.pth

--2021-03-29 09:49:44--  https://download.openmmlab.com/mmaction/recognition/slowfast/slowfast_r50_256p_4x16x1_256e_kinetics400_rgb/slowfast_r50_256p_4x16x1_256e_kinetics400_rgb_20200728-145f1097.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.252.96.35
Connecting to download.openmmlab.com (download.openmmlab.com)|47.252.96.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 138273144 (132M) [application/octet-stream]
Saving to: ‘slowfast.pth’

slowfast.pth        100%[===================>] 131.87M  7.50MB/s    in 16s     

2021-03-29 09:50:00 (8.40 MB/s) - ‘slowfast.pth’ saved [138273144/138273144]



## Build and Import pretrained model

In [1]:
#Check slow_fast.py for implementation details
from slow_fast import slow_fast, hacked_forward

## Code below follow the tutorial

In [2]:
# imports
import argparse
import logging
import time
from tqdm import tqdm
import numpy as np
import torch
from vector_cv_tools import datasets
from vector_cv_tools import transforms as T

from vector_cv_tools import utils

import albumentations as A
from torch.utils.data import DataLoader

import torchvision

In [3]:
def to_loader(dataset, num_workers, batch_size):
    """
    Loads the dataset into a loader for testing
    """
    return DataLoader(dataset,
                      num_workers=num_workers,
                      batch_size=batch_size,
                      collate_fn=utils.VideoCollateFnSelector("stack_or_combine"),
                      shuffle=True)


In [4]:
# define spatial transforms
spatial_transforms = [A.Resize(128, 128), A.ToFloat(max_value=255)]
spatial_transforms = T.ComposeVideoSpatialTransform(spatial_transforms)

# define temporal transforms
temporal_transforms = [ T.video_transforms.RandomTemporalCrop(size=64, 
                                                    pad_if_needed=True,
                                                    padding_mode ="wrap"), 
                        T.video_transforms.ToTensor()]

temporal_transforms = T.ComposeVideoTemporalTransform(temporal_transforms)

print("Spatial transforms: \n{}".format(spatial_transforms))
print("Temporal transforms: \n{}".format(temporal_transforms))


Spatial transforms: 
ComposeVideoSpatialTransform(
    Resize(always_apply=False, p=1, height=128, width=128, interpolation=1)
    ToFloat(always_apply=False, p=1.0, max_value=255)
)
Temporal transforms: 
ComposeVideoTemporalTransform(
    RandomTemporalCrop(size=64, padding=None)
    ToTensor()
)


In [5]:
# create dataset
data = datasets.KineticsDataset(
        fps=10,
        max_frames=128,
        round_source_fps=False,
        annotation_path = "/scratch/ssd002/datasets/kinetics/kinetics700/train.json",
        data_path = "/scratch/ssd002/datasets/kinetics/train",
        class_filter = ["push_up", "pull_ups"],
        spatial_transforms=spatial_transforms,
        temporal_transforms=temporal_transforms,)


In [6]:
# convert data to loader
num_workers = 4
batch_size = 8
loader = to_loader(data, num_workers, batch_size)
labels = data.metadata.labels

print("Looping through the dataset, {} labels, {} data points in total".
        format(data.num_classes, len(loader)))
for label, info in labels.items():
    print("{:<40} ID: {} size: {} {}".
        format(label, info["id"], len(info["indexes"]), len(info["indexes"])//20 * "|"))

Looping through the dataset, 2 labels, 237 data points in total
push_up                                  ID: 0 size: 964 ||||||||||||||||||||||||||||||||||||||||||||||||
pull_ups                                 ID: 1 size: 929 ||||||||||||||||||||||||||||||||||||||||||||||


In [7]:
data_point, label = data[0]
print(data_point.shape)
print(label)
vid = (data_point.numpy() * 255).astype(np.uint8)
utils.create_GIF("TestImage.gif", vid)

torch.Size([64, 128, 128, 3])
{'label_ids': [0], 'label_names': ['push_up'], 'sampled_fps': 10}


In [8]:
# Changed the code here to use slow_fast model
num_classes = data.num_classes
model = slow_fast
print(model)

Recognizer3D(
  (backbone): ResNet3dSlowFast(
    (slow_path): ResNet3dPathway(
      (conv1): ConvModule(
        (conv): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
        (bn): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activate): ReLU(inplace=True)
      )
      (maxpool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), dilation=1, ceil_mode=False)
      (pool2): MaxPool3d(kernel_size=(2, 1, 1), stride=(2, 1, 1), padding=0, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck3d(
          (conv1): ConvModule(
            (conv): Conv3d(80, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
            (bn): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (activate): ReLU(inplace=True)
          )
          (conv2): ConvModule(
            (conv): Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 1

In [11]:
# train the model
model = model.train()
num_epochs = 50
device = torch.device("cuda:0")
model.to(device)

# Need to do that for MMAction2 models
for param in model.parameters():
    param.require_grad=True
for param in model.cls_head.fc_cls.parameters():
    param.require_grad=True

optimizer = torch.optim.Adam(
    model.cls_head.fc_cls.parameters(), 
    lr=0.002)
criterion = torch.nn.CrossEntropyLoss()
losses = []
acc = []
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    start = time.time()
    total = running_corrects = 0
    total_loss = 0
    for idx, (d, l) in enumerate(loader):
        print(idx)
        if idx % 100 == 0:
            name = "KINETICS_VID_{}_{}_LABEL_{}.gif".format(epoch, idx, l[0]["label_names"][0])
            print("Creating GIF:", name)
            vid = (d[0].numpy() * 255).astype(np.uint8)
            utils.create_GIF(name, vid)

        ########### Tweak input ##########
        # depending on what your model wants, tensor shapes may require a permute
        
        #Changed here to adhere to MMAction2 6-dimensional format
        # More on format here 
        #https://github.com/open-mmlab/mmaction2/blob/master/mmaction/datasets/pipelines/formating.py
        d = d.unsqueeze(1)
        inputs = d.to(device).permute(0, 1, 5, 2, 3, 4)
        
        # for single class, we just use the 0th element in the label
        labels = [li["label_ids"][0] for li in l]
        labels = torch.tensor(labels).to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        
        inputs = inputs.reshape((-1, ) + inputs.shape[2:])
        x = model.extract_feat(inputs)
        outputs = model.cls_head(x)
        
        loss = criterion(outputs, labels)
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels)
        total += len(labels)

    duration = time.time() - start
    accuracy = running_corrects / total
    loss =  total_loss / len(loader)
    print("Epoch took {:10.2f}s".format(duration))
    print("Average time per batch {}".format(duration/len(loader)))
    print("Accuracy: ", accuracy)
    print("Epoch Loss: ",loss)
    losses.append(loss)
    acc.append(accuracy)

Epoch 0/49
----------
0
Creating GIF: KINETICS_VID_0_0_LABEL_pull_ups.gif


/pkgs/vector_cv_project/lib/python3.6/site-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
Creating GIF: KINETICS_VID_0_100_LABEL_push_up.gif
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
Creating GIF: KINETICS_VID_0_200_LABEL_push_up.gif
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
Epoch took    2867.69s
Average time per batch 12.0999545366955

/pkgs/vector_cv_project/lib/python3.6/site-packages/vector_cv_tools/utils/video_utils.py:219: UserWarning: Video Loading Failed, stopping iteration                               prematurely
  prematurely")
/pkgs/vector_cv_project/lib/python3.6/site-packages/vector_cv_tools/utils/video_utils.py:219: UserWarning: Video Loading Failed, stopping iteration                               prematurely
  prematurely")
/pkgs/vector_cv_project/lib/python3.6/site-packages/vector_cv_tools/utils/video_utils.py:219: UserWarning: Video Loading Failed, stopping iteration                               prematurely
  prematurely")
/pkgs/vector_cv_project/lib/python3.6/site-packages/vector_cv_tools/utils/video_utils.py:219: UserWarning: Video Loading Failed, stopping iteration                               prematurely
  prematurely")


KeyboardInterrupt: 